In [11]:
import tensorflow as tf
from tensorflow.python.keras import Model, Input
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.optimizers import Adadelta

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np

%matplotlib inline

In [3]:
def parse(serialized):
    # Define a dict with the schema reflecting the data in the TFRecords file
    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        }
    
    # Parse the serialized data
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)
    
    # Get the image as raw bytes
    image_raw = parsed_example['image']
    
    # Convert the raw bytes to tensorflow datatypes
    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    
    # Get the label
    label = parsed_example['label']
    
    # Return the image and label as correct data types
    return image, label
    

In [4]:
def input_fn(filenames, train, batch_size=32, buffer_size=2048):
    # Create a TensorFlow Dataset-object which has functionality for reading and shuffling data 
    # from TFREcords files
    
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Start building the pipeline
    # Parse
    dataset = dataset.map(parse)
    
    # Shuffle when training
    if train:
        dataset = dataset.shuffle(buffer_size = buffer_size)
        # Allow infinite reading of the data
        num_repeat = None
    else:
        num_repeat = 1
        
    # Repeat the dataset the given number of times
    dataset = dataset.repeat(num_repeat)
    
    # Set the batch size
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    images_batch, labels_batch = iterator.get_next()
    
    return {'image':images_batch}, labels_batch
    
    

In [5]:
path_tfrecords_train = '../data/tfrecords/training.tfrecords'
def train_input_fn():
    return input_fn(filenames=path_tfrecords_train, train=True)

In [6]:
path_tfrecords_test = '../data/tfrecords/validation.tfrecords'
def test_input_fn():
    return input_fn(filenames=path_tfrecords_test, train=False)

In [7]:
image1 = np.array(Image.open('../data/validation/st1179_sound_9.PNG'))
image2 = np.array(Image.open('../data/validation/st1179_dry_knot_3.PNG'))
some_images = np.array([image1, image2])/255
some_images = np.reshape(some_images, (2, 37632))


In [8]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": some_images.astype(np.float32)},
    num_epochs=1,
    shuffle=False)

In [ ]:
y_train = tf.keras.utils.to_categorical(labels, 7)
x_train = images/255

inputs = Input(shape=(112, 112, 3))
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
y = Dense(7, activation='softmax')(x)

model = Model(inputs=inputs, outputs=y)

model.compile(optimizer = Adadelta(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])



In [9]:
params = {"learning_rate": 1e-4}

In [10]:
odel = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir="../checkpoints")

NameError: name 'model_fn' is not defined

In [ ]:
model.train(input_fn=train_input_fn, steps=8000)


In [ ]:
result = model.evaluate(input_fn=test_input_fn)

In [ ]:
predictions = model.predict(input_fn=predict_input_fn)

In [ ]:
cls_pred = np.array(list(predictions))
cls_pred